In [ ]:
dbutils.widgets.text("SOURCE_CATALOG", "")
source_catalog = dbutils.widgets.get("SOURCE_CATALOG")

In [ ]:
spark.sql(f'''
CREATE OR REPLACE VIEW {source_catalog}.datasteward.m_metadata_quality_descriptions
AS (
  WITH description_settings AS (
    SELECT
      LOWER(catalog)                AS catalog_name,
      LOWER(schema)                 AS schema_name,
      LOWER(COALESCE(table, ""))    AS table_name,
      COALESCE(name_ja, "")         AS name_ja,
      table_category                AS table_category,
      COALESCE(cron_schedule, "")   AS cron_schedule,
      COALESCE(explanation, "")     AS explanation,
      COALESCE(type_conversion, "") AS type_conversion,
      COALESCE(rule, "")            AS rule,
      COALESCE(link, "")            AS link,
      COALESCE(query, "")           AS query,
      COALESCE(reference, "")       AS reference
    FROM
      dataplatform_public_published_dev.datasteward.description_settings
  ),
  actual_description AS (
    SELECT
      LOWER(catalog_name) AS catalog_name,
      LOWER(schema_name)  AS schema_name,
      ""                  AS table_name,
      0                   AS table_category,
      comment             AS description
    FROM
      system.information_schema.schemata
    WHERE TRUE
      AND catalog_name NOT IN ("system", "samples")
      AND schema_name NOT IN ("information_schema")

    UNION ALL

    SELECT
      LOWER(table_catalog) AS catalog_name,
      LOWER(table_schema)  AS schema_name,
      LOWER(table_name)    AS table_name,
      CASE 
        WHEN table_type IN ("MANAGED", "STREAMING_TABLE", "EXTERNAL") THEN 1
        WHEN table_type = "MATERIALIZED_VIEW" THEN 2
        WHEN table_type = "VIEW" THEN 3
        ELSE NULL
      END AS table_category,
      comment             AS description
    FROM
      system.information_schema.tables
    WHERE TRUE
      AND table_catalog NOT IN ("system", "samples")
      AND table_schema NOT IN ("information_schema")
  ),
  current_descriptions AS (
    SELECT *, dataplatform_public_published_dev.datasteward.generate_description_schema(name_ja, explanation) AS description
    FROM description_settings
    WHERE table_category = 0

    UNION ALL

    SELECT *, dataplatform_public_published_dev.datasteward.generate_description_table(name_ja, explanation, type_conversion, rule, cron_schedule, query, reference, link) AS description
    FROM description_settings
    WHERE table_category = 1 AND catalog_name LIKE "%staging%"

    UNION ALL

    SELECT *, dataplatform_public_published_dev.datasteward.generate_description_table(name_ja, explanation, type_conversion, rule, cron_schedule, query, reference, "") AS description
    FROM description_settings
    WHERE table_category = 1 AND (catalog_name LIKE "%conformed%" OR catalog_name LIKE "%published%")

    UNION ALL

    SELECT *, dataplatform_public_published_dev.datasteward.generate_description_view(name_ja, cron_schedule, reference) AS description
    FROM description_settings
    WHERE table_category IN (2, 3)
  )
  SELECT 
    COALESCE(actual_description.catalog_name, current_descriptions.catalog_name) AS catalog_name,
    COALESCE(actual_description.schema_name, current_descriptions.schema_name) AS schema_name,
    COALESCE(actual_description.table_name, current_descriptions.table_name) AS table_name,
    COALESCE(actual_description.table_category, current_descriptions.table_category) AS table_category,
    current_descriptions.name_ja,
    current_descriptions.cron_schedule,
    current_descriptions.explanation,
    current_descriptions.type_conversion,
    current_descriptions.rule,
    current_descriptions.link,
    current_descriptions.query,
    current_descriptions.reference,
    actual_description.description AS actual_description,
    current_descriptions.description AS current_description,
    CASE
      WHEN COALESCE(actual_description.description, "") = COALESCE(current_descriptions.description, "") THEN 0
      ELSE 1
    END AS flg_diff,
    CONCAT(DATE_FORMAT(CONVERT_TIMEZONE("UTC","Asia/Tokyo", CURRENT_TIMESTAMP()), "yyyy-MM-dd HH:mm"), "（JST）時点") AS refreshed_at_jst
  FROM actual_description
    FULL OUTER JOIN current_descriptions
      ON actual_description.catalog_name = current_descriptions.catalog_name
      AND actual_description.schema_name = current_descriptions.schema_name
      AND actual_description.table_name = current_descriptions.table_name
      AND actual_description.table_category = current_descriptions.table_category
  ORDER BY
    catalog_name,
    schema_name,
    table_name,
    table_category,
    flg_diff
)
''')

In [ ]:
spark.sql(f'''
CREATE OR REPLACE VIEW {source_catalog}.datasteward.m_metadata_quality_tags
AS (
  WITH current_tags AS (
    SELECT
      LOWER(catalog)                AS catalog_name,
      LOWER(schema)                 AS schema_name,
      LOWER(COALESCE(table, ""))    AS table_name,
      table_category                AS table_category,
      COALESCE(tag_key, "")         AS tag_key,
      COALESCE(tag_value, "")       AS tag_value
    FROM
      dataplatform_public_published_dev.datasteward.tag_settings
  ),
  actual_tags AS (
    SELECT
      LOWER(catalog_name) AS catalog_name,
      LOWER(schema_name)  AS schema_name,
      ""                  AS table_name,
      0                   AS table_category,
      tag_name            AS tag_key,
      tag_value           AS tag_value
    FROM
      system.information_schema.schema_tags
    WHERE TRUE
      AND catalog_name NOT IN ("system", "samples")
      AND schema_name  NOT IN ("information_schema")

    UNION ALL

    SELECT
      LOWER(tags.catalog_name) AS catalog_name,
      LOWER(tags.schema_name)  AS schema_name,
      LOWER(tags.table_name)   AS table_name,
      CASE 
        WHEN tables.table_type IN ("MANAGED", "STREAMING_TABLE", "EXTERNAL") THEN 1
        WHEN tables.table_type = "MATERIALIZED_VIEW" THEN 2
        WHEN tables.table_type = "VIEW" THEN 3
        ELSE NULL
      END AS table_category,
      tags.tag_name            AS tag_key,
      tags.tag_value           AS tag_value
    FROM
      system.information_schema.table_tags AS tags
      LEFT JOIN system.information_schema.tables tables
      ON tags.catalog_name = tables.table_catalog
      AND tags.schema_name = tables.table_schema
      AND tags.table_name = tables.table_name
    WHERE TRUE
      AND catalog_name NOT IN ("system", "samples")
      AND schema_name  NOT IN ("information_schema")
  )
  SELECT
    COALESCE(actual_tags.catalog_name, current_tags.catalog_name) AS catalog_name,
    COALESCE(actual_tags.schema_name, current_tags.schema_name) AS schema_name,
    COALESCE(actual_tags.table_name, current_tags.table_name) AS table_name,
    COALESCE(actual_tags.table_category, current_tags.table_category) AS table_category,
    actual_tags.tag_key AS actual_tag_key,
    actual_tags.tag_value AS actual_tag_value,
    current_tags.tag_key AS current_tag_key,
    current_tags.tag_value AS current_tag_value,
    CASE
      WHEN (actual_tags.tag_key = current_tags.tag_key) AND (actual_tags.tag_value = current_tags.tag_value) THEN 0
      ELSE 1
    END AS flg_diff,
    CONCAT(DATE_FORMAT(CONVERT_TIMEZONE("UTC","Asia/Tokyo", CURRENT_TIMESTAMP()), "yyyy-MM-dd HH:mm"), "（JST）時点") AS refreshed_at_jst
  FROM actual_tags
    FULL OUTER JOIN current_tags
    ON actual_tags.catalog_name = current_tags.catalog_name
    AND actual_tags.schema_name = current_tags.schema_name
    AND actual_tags.table_name = current_tags.table_name
    AND actual_tags.table_category = current_tags.table_category
    AND actual_tags.tag_key = current_tags.tag_key
  ORDER BY
    catalog_name,
    schema_name,
    table_name,
    table_category,
    flg_diff
)
''')